In [1]:
import cx_Oracle
import csv
import io

In [2]:
cx_Oracle.init_oracle_client(lib_dir="your_dir")

In [3]:
conn = cx_Oracle.connect(r'your_login', r'your_password', 'db202009151316_high')

In [4]:
cursor = conn.cursor()

In [6]:
cursor.execute("CREATE TABLE PHONES2(name VARCHAR2(100),phnumber VARCHAR2(40),phnumber_int NUMBER,inn NUMBER)")

In [7]:
def clearNumber(phone):
    if (phone == ""):
        return 0
    rem = " +()-"
    for char in rem:
        phone = phone.replace(char, "")
    numPh = int(phone)
    return numPh

In [8]:
path = "file.csv"
with io.open(path, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=';')
    first = True
    for r in reader:
        if first:
            first = False
            continue
        res = clearNumber(r[4])
        cursor.execute("INSERT INTO PHONES2(name, phnumber, phnumber_int, inn) VALUES(:name, :phnumber, :phnumber_int, :inn)", [r[3], r[4], res, r[7]])
        conn.commit()

In [9]:
cursor.execute("SELECT * FROM PHONES2")
result = cursor.fetchall()
print(result)

[('Филимошин Роман Владимирович', '7 (495) 913-09-65', 74959130965, 1024000939643), ('Мелешкин Михаил Федорович', None, 0, 1037700000095), ('Фесенко Павел Федорович', '7 (8622) 90-72-02', 78622907202, 1022302934873), ('Моcиенко Сергей Викторович', '7 (861) 33-3-35-82', 78613333582, 1022300513388), ('Лисицына Лариса Львовна', '7 (495) 577-95-11', 74955779511, 1165029054530), ('Матвиенко Алексей Павлович', None, 0, 1022502131190), ('Грищенко Юрий Владимирович', '7 10 (38065) 424-72-30', 710380654247230, 1026101345445), ('Тришина Елена Александровна', '7 (496) 31-721-00', 74963172100, 1025001820645), ('Беляков Николай Федорович', '7 (831) 437-08-30', 78314370830, 1025203041160), ('Мурашов Сергей Борисович', None, 0, 1027807590106)]


In [25]:
cursor.execute("CREATE TABLE FULL(phnumber VARCHAR2(40),inn NUMBER)")

In [26]:
path = "file-full.csv"
with io.open(path, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=';')
    first = True
    for r in reader:
        if first:
            first = False
            continue
        cursor.execute("INSERT INTO FULL(phnumber, inn) VALUES(:phnumber, :inn)", [r[4], r[7]])
        conn.commit()

In [27]:
cursor.execute("SELECT * FROM FULL")
result = cursor.fetchall()
print(result)

[('7 (495) 913-09-65', 1024000939643), ('7 (495) 606-71-94', 1037700000095), ('7 (8622) 90-72-02', 1022302934873), ('7 (861) 33-3-35-82', 1022300513388), ('7 (495) 577-95-11', 1165029054530), ('7 (4232) 24-02-56', 1022502131190), ('7 10 (38065) 424-72-30', 1026101345445), ('7 (496) 31-721-00', 1025001820645), ('7 (831) 437-08-30', 1025203041160), ('7 (812) 492-20-58', 1027807590106)]


In [11]:
cursor.execute("UPDATE PHONES2 SET phnumber = (SELECT phnumber FROM FULL WHERE PHONES2.inn = FULL.inn) WHERE phnumber IS NULL")

In [12]:
cursor.execute("SELECT * FROM PHONES2")
result = cursor.fetchall()
print(result)

[('Филимошин Роман Владимирович', '7 (495) 913-09-65', 74959130965, 1024000939643), ('Мелешкин Михаил Федорович', '7 (495) 606-71-94', 0, 1037700000095), ('Фесенко Павел Федорович', '7 (8622) 90-72-02', 78622907202, 1022302934873), ('Моcиенко Сергей Викторович', '7 (861) 33-3-35-82', 78613333582, 1022300513388), ('Лисицына Лариса Львовна', '7 (495) 577-95-11', 74955779511, 1165029054530), ('Матвиенко Алексей Павлович', '7 (4232) 24-02-56', 0, 1022502131190), ('Грищенко Юрий Владимирович', '7 10 (38065) 424-72-30', 710380654247230, 1026101345445), ('Тришина Елена Александровна', '7 (496) 31-721-00', 74963172100, 1025001820645), ('Беляков Николай Федорович', '7 (831) 437-08-30', 78314370830, 1025203041160), ('Мурашов Сергей Борисович', '7 (812) 492-20-58', 0, 1027807590106)]


In [13]:
conn.close()